In [1]:
import sys
import numpy as np
import xarray as xr

sys.path.append(".")
from automatic_setup import wrapper_zoo_model
from seapopym.configuration.no_transport.parameter import (
    ForcingParameters,
    EnvironmentParameter,
)
from dask.distributed import Client, LocalCluster
from seapopym.configuration.parameters.parameter_environment import ClientParameter
from seapopym.configuration.parameters.parameter_forcing import ForcingUnit
from seapopym.logging.custom_logger import set_error
from seapopym.standard.units import StandardUnitsLabels
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import numpy as np
import random
import plotly.express as px
import pandas as pd

random.seed(64)

In [2]:
set_error()
xr.set_options(
    keep_attrs=True,
    display_expand_attrs=False,
    display_expand_data=False,
    display_expand_coords=False,
    display_expand_data_vars=False,
)

WET_TO_CARBON = 1 / 11.9

## Forcing


Forcing data from seapodym model


In [3]:
hot_data = xr.open_dataset(
    "../1_data_processing/1_1_Forcing/hot_cmems.zarr", engine="zarr"
)
hot_data["T"].attrs["units"] = StandardUnitsLabels.temperature.units
hot_data.load()

<xarray.Dataset> Size: 829kB
Dimensions:              (time: 9011, depth: 3, latitude: 1, longitude: 1)
Coordinates: (4)
Data variables: (13)
Attributes: (10)

Observation data from Hot station


In [4]:
hot_obs = xr.open_dataset(
    "../1_data_processing/1_1_Forcing/hot_obs.zarr", engine="zarr"
)
hot_obs.load()

<xarray.Dataset> Size: 2MB
Dimensions:        (time: 659, latitude: 1, longitude: 1, depth: 8, frac: 5)
Coordinates: (5)
Data variables: (24)

Generate the observed zooplankton data to be used further in the cost function


In [5]:
zoo_obs = (
    (hot_obs["dwt_migrant"] + hot_obs["dwt_resident"])
    .sum("frac")
    .sum("depth")
    .cf.isel(Y=0, X=0)
    .pint.quantify()
    .pint.to("kg/m^2")
)
zoo_obs = zoo_obs.where(zoo_obs > 0, drop=True) * WET_TO_CARBON
zoo_obs

<xarray.DataArray (time: 458)> Size: 4kB
[kg/m²] 6.136e-05 9.605e-06 4.229e-05 ... 0.0002705 8.388e-05 0.0001412
Coordinates: (3)
Attributes: (36)

## Setup


The forcing data


In [6]:
hot_data_parameter = ForcingParameters(
    temperature=ForcingUnit(forcing=hot_data["T"], resolution=0.08333),
    primary_production=ForcingUnit(forcing=hot_data["npp"], resolution=0.08333),
)

The initial conditions


In [7]:
setup_model = wrapper_zoo_model(
    tr_max=10.38,
    tr_rate=-0.11,
    inv_lambda_max=150,
    inv_lambda_rate=0.15,
    forcing_parameter=hot_data_parameter,
)
initial_conditions = setup_model.export_initial_conditions()

Update the forcing data with initial conditions


In [8]:
hot_data_parameter = ForcingParameters(
    temperature=ForcingUnit(forcing=hot_data["T"], resolution=0.08333),
    primary_production=ForcingUnit(forcing=hot_data["npp"], resolution=0.08333),
    initial_condition_biomass=ForcingUnit(
        forcing=initial_conditions["initial_condition_biomass"], resolution=0.08333
    ),
    initial_condition_production=ForcingUnit(
        forcing=initial_conditions["initial_condition_production"], resolution=0.08333
    ),
)

Setup the kernel to use multiple processors


In [9]:
cluster = LocalCluster(n_workers=4, threads_per_worker=1, memory_limit="2GB")
client = Client(cluster)
client_param = ClientParameter.from_address(address=client.scheduler.address)
client_param.client

<Client: 'tcp://127.0.0.1:58287' processes=4 threads=4, memory=7.45 GiB>

## Cost function


In [10]:
def cost_function(args: tuple[float, float, float, float]):
    """Use the Mean Absolute Error (MAE) method."""

    tr_max, tr_rate, inv_lambda_max, inv_lambda_rate = args

    setup_model = wrapper_zoo_model(
        tr_max=tr_max,
        tr_rate=tr_rate,
        inv_lambda_max=inv_lambda_max,
        inv_lambda_rate=inv_lambda_rate,
        forcing_parameter=hot_data_parameter,
        environment=EnvironmentParameter(client=client_param),
    )
    biomass_pred = (
        setup_model.export_biomass()
        .cf.isel(functional_group=0, X=0, Y=0)
        .pint.quantify()
    )
    cost = float(np.abs(zoo_obs - biomass_pred).mean())
    return (cost,)

## DEAP - Genetic algorithm


Create the population of the genetic algorithm.


In [11]:
def random_exclusive(a, b):
    if a >= b:
        raise ValueError(
            "La borne inférieure doit être inférieure à la borne supérieure."
        )

    while True:
        value = random.uniform(a, b)
        if value != a and value != b:
            return value


parameters_boundaries = {
    "tr_max": (0, 50),
    "tr_rate": (-1, 0),
    "inv_lambda_max": (1, 1000),
    "inv_lambda_rate": (0, 1),
}
lower_boundary = [values[0] for values in parameters_boundaries.values()]
upper_boundary = [values[1] for values in parameters_boundaries.values()]

ETA = 0.1
INDPB = 0.05


def mutation(individual):
    return tools.mutPolynomialBounded(
        individual, ETA, lower_boundary, upper_boundary, INDPB
    )

In [12]:
toolbox = base.Toolbox()

creator.create("Fitness", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.Fitness)

for parameter, boundaries in parameters_boundaries.items():
    lower_boundary, upper_boundary = boundaries
    toolbox.register(parameter, random_exclusive, lower_boundary, upper_boundary)

toolbox.register(
    "individual",
    tools.initCycle,
    creator.Individual,
    (toolbox.tr_max, toolbox.tr_rate, toolbox.inv_lambda_max, toolbox.inv_lambda_rate),
    n=1,
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
number_of_individuals = 10

And now the cost function and behavior of the genetic algorithm.


In [13]:
toolbox.register("evaluate", cost_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutation)
toolbox.register("select", tools.selTournament, tournsize=3)

The main function to run the genetic algorithm.


In [14]:
# def main():
#     pop = toolbox.population(n=number_of_individuals)
#     CXPB, MUTPB, NGEN = 0.5, 0.2, 10

#     # Evaluate the entire population
#     fitnesses = list(map(toolbox.evaluate, pop))
#     for ind, fit in zip(pop, fitnesses):
#         ind.fitness.values = fit

#     for g in range(NGEN):
#         print("-- Generation %i --" % g)
#         print(pop)
#         # Select the next generation individuals
#         offspring = toolbox.select(pop, len(pop))
#         # Clone the selected individuals
#         offspring = list(map(toolbox.clone, offspring))

#         # Apply crossover and mutation on the offspring
#         for child1, child2 in zip(offspring[::2], offspring[1::2]):
#             if random.random() < CXPB:
#                 toolbox.mate(child1, child2)
#                 del child1.fitness.values
#                 del child2.fitness.values

#         for mutant in offspring:
#             if random.random() < MUTPB:
#                 toolbox.mutate(mutant)
#                 del mutant.fitness.values

#         # Evaluate the individuals with an invalid fitness
#         invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
#         print(invalid_ind)
#         fitnesses = list(map(toolbox.evaluate, invalid_ind))
#         for ind, fit in zip(invalid_ind, fitnesses):
#             ind.fitness.values = fit

#         # The population is entirely replaced by the offspring
#         pop[:] = offspring

#     return pop

In [15]:
def main(ngen=10, npop=10):
    pop = toolbox.population(n=npop)
    hof = tools.HallOfFame(10)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    pop, log = algorithms.eaSimple(
        pop,
        toolbox,
        cxpb=0.5,
        mutpb=0.2,
        ngen=ngen,
        stats=stats,
        halloffame=hof,
        verbose=True,
    )
    return pop, log, hof

In [16]:
pop, log, hof = main()

2024-06-26 15:05:49,165 - distributed.worker - WARNING - Compute Failed
Key:       ('_production_helper-a8fd88197d268ce6022c6ab4222ebf8d', 0, 0, 0, 0, 0, 0)
State:     executing
Function:  execute_task
args:      ((<function map_blocks.<locals>._wrapper at 0x10e0393a0>, <function _production_helper at 0x10d49b560>, [(<class 'xarray.core.dataset.Dataset'>, (<class 'dict'>, [[<ConfigurationLabels.fgroup_name: 'name'>, ((<CoordinatesLabels.functional_group: 'functional_group'>,), array(['D1N1'], dtype='<U4'), {'description': 'The name of the functional group.'})], [<ConfigurationLabels.energy_transfert: 'energy_transfert'>, ((<CoordinatesLabels.functional_group: 'functional_group'>,), array([0.1668]), {'description': 'Energy transfert coefficient between primary production and functional group.'})], [<ConfigurationLabels.inv_lambda_max: 'inv_lambda_max'>, ((<CoordinatesLabels.functional_group: 'functional_group'>,), array([157.26903049]), {'description': 'Value of 1/lambda when temperatur

ValueError: Received dimension 'cohort' of length 11. Expected length 44.

In [ ]:
all_res = []
for index, individual in enumerate(hof.items):
    pred_biomass = wrapper_zoo_model(
        tr_max=individual[0],
        tr_rate=individual[1],
        inv_lambda_max=individual[2],
        inv_lambda_rate=individual[3],
        forcing_parameter=hot_data_parameter,
    )
    pred_biomass = (
        pred_biomass.export_biomass()
        .cf.isel(functional_group=0, X=0, Y=0)
        .pint.quantify()
    )
    pred_biomass = (
        pred_biomass.to_dataframe()
        .reset_index()
        .drop(columns=["latitude", "longitude", "functional_group"])
        .rename(columns={"biomass": f"sim_{index}"})
    )
    all_res.append(pred_biomass.set_index("time"))

In [ ]:
all_res.append(
    zoo_obs.rename("observation")
    .to_dataframe()
    .drop(columns=["latitude", "longitude"])["observation"]
)

In [ ]:
results = pd.concat(all_res, axis=1)
results = (
    results.stack()
    .reset_index()
    .rename(columns={"level_1": "simulation", 0: "biomass"})
)
results

In [ ]:
fig = px.line(
    results,
    x="time",
    y="biomass",
    color="simulation",
    title="Simulation results",
)
fig.show()